#### ACM SIGCHI 2017 Course on Computational Interaction


# Probabilistic filtering
### <font color="green"> Inferring user intention in a noisy world </font>
 
#### <b>John Williamson</b> / University of Glasgow

[relevant pretty pictures here]

--------------------------

## Introduction [7 mins]
### What is probabilistic filtering?
[ELI5 two or three sentences here.]
### Why is this computational HCI?
[and not just plain old HCI]
### What are existing techniques?
[i.e. non-computational techniques and why they aren't as good]
### Benefits
[What can be acheived that was not possible before]
### History
[how did this come about, maybe in two or three papers]

## Principles [28 mins]
### Overview diagram

### Use case
### Problem description
We are going to solve xxx
### Meat
Meat goes here!

### Gallery
Research papers here (thumbnail + link), short description of why cool

### Pitfalls
Hands-on guru knowledge goes here.

## Outlook [5 mins]
### Scope and limitations

### Resources
How do I learn to do this in depth? Blogs, papers, software, videos, online courses, etc.

### Future
How will this evolve over the next few years.